In [16]:
import tkinter as tk
from tkinter import ttk


In [ ]:
root = tk.Tk()
root.geometry("600x400")
root.title("Face Recognizer")
root.configure(bg="gray")
root.resizable(False, False)

label = tk.Label(root, text="WELCOME TO FACE RECOGNIZER...", font=('Times New Roman', 20, 'bold'), bg="gray")
label.pack(padx=40, pady=20)

# Add a label for camera selection
camera_label = tk.Label(root, text="Select Camera:", font=('Times New Roman', 15, 'bold'), bg="gray")
camera_label.pack(pady=10)

# Add a dropdown (combobox) for selecting webcam or external cam
camera_options = ["Webcam", "External Cam"]
camera_select = ttk.Combobox(root, values=camera_options, font=('Georgia', 14), state="readonly")
camera_select.current(0)  # Set default to 'Webcam'
camera_select.pack(pady=20)

# Login button
login_button = tk.Button(root, text="Detect Face", font=('Georgia', 14, 'bold'), bg="white")
login_button.pack(pady=10)

label = tk.Label(root, text="or", font=('Times New Roman', 15, 'bold'), bg="gray")
label.pack(padx=5, pady=5)

# Add Face button
create_button = tk.Button(root, text="Add Face", font=('Georgia', 14, 'bold'), bg="white")
create_button.pack()

root.mainloop()

In [1]:
import cv2

In [2]:

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:  # Check if no faces are detected
            return None
        
        for (x, y, w, h) in faces:  # Correct w and h instead of x and h
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face  # Return cropped face

    cap = cv2.VideoCapture(0)
    id = 1
    img_id = 0

    while True:
        ret, frame = cap.read()
        cropped = face_cropped(frame)  # Call face_cropped and store in 'cropped'
        
        if cropped is not None:  # Only proceed if a face is detected
            img_id += 1
            face = cv2.resize(cropped, (200, 200))  # Resize the cropped face
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            file_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(file_path, face)

            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("Cropped face", face)
            if cv2.waitKey(1) == 13 or int(img_id) == 1000:
                break

    cap.release()
    cv2.destroyAllWindows()
    print("Completed...")

generate_dataset()


Completed...


In [3]:
!pip install numpy

In [4]:
!pip install pillow

In [5]:
!pip install opencv-contrib-python --user


In [6]:
import numpy as np
from PIL import Image
import os
import cv2

In [7]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Convert to grayscale
        imageNp = np.array(img, 'uint8')  # Convert image to numpy array
        id = int(os.path.split(image)[1].split(".")[1])  # Extract ID from image filename

        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)

    # Initialize the LBPH face recognizer
    clf = cv2.face.LBPHFaceRecognizer_create()
    
    # Train the classifier on the faces and IDs
    clf.train(faces, ids)
    
    # Save the trained model to a file
    clf.write("classifier.xml")

# Call the function
train_classifier("data")


In [8]:
!pip install opencv-contrib-python --user


In [9]:
import cv2
import numpy as np
from PIL import Image
import os

In [11]:
def draw_boundary(image, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)

    coords = []
    for (x, y, w, h) in features:
        cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
        
        # Only predict if the region is valid and contains enough pixels
        if w > 0 and h > 0:
            id, pred = clf.predict(gray_image[y:y+h, x:x+w])  # Correct slicing here
            confidence = int(100 * (1 - pred / 300))
            
            if confidence > 72:
                if id == 1:
                    cv2.putText(image, "poorna", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(image, "unknown", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)

        coords = [x, y, w, h]
    return coords

def recognize(img, classifier, faceCascade):
    coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "face", clf)
    return img

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    img = recognize(img, clf, faceCascade)
    cv2.imshow("face detection", img)

    if cv2.waitKey(1) == 13:  # Press Enter to exit
        break

video_capture.release()
cv2.destroyAllWindows()
